In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# import os

from datetime import datetime
import time

In [2]:
key_words = ["thị%20trường%20chứng%20khoán", "vn30", "thị%20trường%20tài%20chính", "vn-index"]
news_time_list = []
news_headline_list = []
news_subtitle_list = []
combined_news_list = []
for key_word in key_words:
    year = 2024
    for i in list(range(1,100)):
        chrome_options = Options()
        chrome_options.add_argument("--headless=new") # for Chrome >= 109

        # Set the download directory preference
        # download_directory = os.path.abspath("C:/Users/Admin/Downloads/News data scraping")
        # prefs = {"download.default_directory": download_directory}
        # chrome_options.add_experimental_option("prefs", prefs)
        website = f"https://cafebiz.vn/timelinesearch/{key_word}/{i}.htm"
        # path = "C/Users/Admin/Documents/Old D disk/Web scraping/"
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(website)
        driver.maximize_window()

        time.sleep(2)
        all_news = driver.find_elements(By.XPATH, '//div[@class="infodsthome"]')
        
        if not all_news:
            driver.quit()
            # If no news found, break out of the loop
            break
        for news in all_news:
            published_time = news.find_element(By.XPATH, "./p[@class='cate']").text
            # Convert the time string to a datetime object
            published_time = datetime.strptime(published_time, "%d/%m/%Y - %H:%M")
            year = int(published_time.year)
            if year <= 2023:
                break

            news_headline = news.find_element(By.XPATH, './h3').text
            news_subtitle = news.find_element(By.XPATH, "./p[@class='sapo']").text

            # Combine headline and subtitle and add to the new list
            combined_news_list.append(f"{news_headline}: {news_subtitle}")

            # Add the unique news entry to your lists
            news_time_list.append(published_time)
            news_headline_list.append(news_headline)
            news_subtitle_list.append(news_subtitle)

        if year <= 2023:
            driver.quit()
            break

        print(news_time_list[-1])
        print(year)

        driver.quit()

In [3]:
df = pd.DataFrame({'Published_time': news_time_list, 'news_content': combined_news_list, 'news_headline': news_headline_list})
df.to_csv('cafebiz/News_headlines_cafebiz_data_update.csv', index=False, encoding='utf-8-sig')


In [5]:
import json
from dateutil import parser

key_words = ["thị%20trường%20chứng%20khoán", "thị%20trường%20tài%20chính"]
news_time_list = []
news_headline_list = []
news_subtitle_list = []
combined_news_list = []

for key_word in key_words:
    year = 2024
    for i in list(range(1,100)):
        chrome_options = Options()
        chrome_options.add_argument("--headless=new") # for Chrome >= 109

        # Set the download directory preference
        # download_directory = os.path.abspath("C:/Users/Admin/Downloads/News data scraping")
        # prefs = {"download.default_directory": download_directory}
        # chrome_options.add_experimental_option("prefs", prefs)
        website = f"https://search.hemera.com.vn/search/1/{key_word}/time/{i}"
        # path = "C/Users/Admin/Documents/Old D disk/Web scraping/"
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(website)
        driver.maximize_window()

        time.sleep(2)

        # Extract title, PublishedDate, and Sapo
        all_news = driver.find_element(By.XPATH, '//body').text
        if not all_news:
            driver.quit()
            # If no news found, break out of the loop
            break

        # Load JSON data
        all_news = json.loads(all_news)

        for item in all_news["List"]:
            # Extracting information
            published_time = item["PublishedDate"]
            published_time = parser.parse(published_time)
            published_time = published_time.replace(microsecond=0).replace(tzinfo=None)

            year = int(published_time.year)
            if year <= 2023:
                break
            news_headline = item["Title"]
            news_subtitle = item["Sapo"]

            # Combine headline and subtitle and add to the new list
            combined_news_list.append(f"{news_headline}: {news_subtitle}")
            # Add the unique news entry to your lists
            news_time_list.append(published_time)
            news_headline_list.append(news_headline)
            news_subtitle_list.append(news_subtitle)

        if year <= 2023:
            driver.quit()
            break
        print(news_time_list[-1])
        print(year)

        driver.quit()
df = pd.DataFrame({'Published_time': news_time_list, 'news_content': combined_news_list, 'news_headline': news_headline_list})
df.to_csv('vneconomy/News_headlines_vneconomy_data_update.csv', index=False, encoding='utf-8-sig')

2024-01-24 02:02:48
2024


In [16]:
import re
news_time_list = []
news_headline_list = []
news_subtitle_list = []
combined_news_list = []
year = 2024

chrome_options = Options()
# chrome_options.add_argument("--headless=new") # for Chrome >= 109

website = f"https://vietstock.vn/chung-khoan.htm"
# path = "C/Users/Admin/Documents/Old D disk/Web scraping/"
driver = webdriver.Chrome(options=chrome_options)
driver.get(website)
driver.maximize_window()
daterange = driver.find_element(By.XPATH, '//input[@name="daterange"]')
daterange.clear()
daterange.send_keys("2020-01-01 - 2021-01-21")

while year > 2019:

    time.sleep(2)
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//div[@id="channel-container"]')))
    all_news = driver.find_elements(By.XPATH, '//div[@id="channel-container"]')

    if not all_news:
        driver.quit()
        # If no news found, break out of the loop
        break
    for news in all_news:
        news = news.find_element(By.XPATH, './/div[@class="single_post_text"]')
        news_headline = news.find_element(By.XPATH, './/h4').text
        temp_element = news.find_element(By.XPATH, ".//a[2]")

        # Extract the href attribute and text content from the element
        href_attribute = temp_element.get_attribute('href')
        published_day_month = temp_element.text
        
        match = re.search(r'/(\d{4})/', href_attribute)
        published_year = int(match.group(1)) if match else None

        # Combine the extracted year with the text content
        combined_datetime_str = f"{published_year}/{published_day_month}"
        try:
            # Convert the combined string to a datetime object
            published_time = datetime.strptime(combined_datetime_str, "%Y/%d/%m %H:%M")
        except:
            published_time = datetime.now()

        year = int(published_time.year)
        if year <= 2019:
            break

        news_subtitle = news.find_element(By.XPATH, ".//p").text

        # Combine headline and subtitle and add to the new list
        combined_news_list.append(f"{news_headline}: {news_subtitle}")

        # Add the unique news entry to your lists
        news_time_list.append(published_time.strftime("%Y-%m-%d %H:%M:%s"))
        news_headline_list.append(news_headline)
        news_subtitle_list.append(news_subtitle)

    if year <= 2019:
        driver.quit()
        break
    
    time.sleep(5)
    try:
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, ".//li[@id='page-next ']/a")))
        next_page = driver.find_element(By.XPATH, ".//li[@id='page-next ']/a")
        next_page.click()
    except:
        time.sleep(8)
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, ".//li[@id='page-next ']/a")))
        next_page = driver.find_element(By.XPATH, ".//li[@id='page-next ']/a")
        next_page.click()
    print(news_time_list[-1])
    print(year)

driver.quit()

2024-02-26 11:58
2024
2021-01-20 10:24
2021
2021-01-19 20:00
2021
2021-01-19 10:00
2021
2021-01-18 19:30
2021
2021-01-18 11:04
2021
2021-01-17 19:57
2021
2021-01-16 10:30
2021
2021-01-15 10:58
2021
2021-01-14 19:30
2021
2021-01-14 08:45
2021
2021-01-13 15:35
2021
2021-01-13 06:00
2021
2021-01-12 13:00
2021
2021-01-11 20:00
2021
2021-01-11 13:05
2021
2021-01-11 06:00
2021
2021-01-09 10:54
2021
2021-01-08 13:00
2021
2021-01-07 19:30
2021
2021-01-07 09:33
2021
2021-01-06 16:20
2021
2021-01-06 08:39
2021
2021-01-05 15:40
2021
2021-01-05 09:00
2021
2021-01-04 17:12
2021
2021-01-04 13:09
2021
2021-01-04 08:51
2021
2021-01-03 09:00
2021
2021-01-01 12:00
2021
2020-12-31 10:00
2020
2020-12-30 17:22
2020
2020-12-30 10:34
2020
2020-12-29 17:10
2020
2020-12-29 11:10
2020
2020-12-28 20:00
2020
2020-12-28 13:58
2020
2020-12-28 08:38
2020
2020-12-26 19:45
2020
2020-12-25 13:45
2020
2020-12-24 19:30
2020
2020-12-24 11:15
2020
2020-12-23 20:00
2020
2020-12-23 11:14
2020
2020-12-22 17:06
2020
2020-12-22

In [17]:
df = pd.DataFrame({'Published_time': news_time_list, 'news_content': combined_news_list, 'news_headline': news_headline_list})
df.to_csv('vietstock/News_headlines_vietstock_data_4.csv', index=False, encoding='utf-8-sig')

In [15]:
driver.quit()

In [9]:
chrome_options = Options()
# chrome_options.add_argument("--headless=new") # for Chrome >= 109

# Set the download directory preference
# download_directory = os.path.abspath("C:/Users/Admin/Downloads/News data scraping")
# prefs = {"download.default_directory": download_directory}
# chrome_options.add_experimental_option("prefs", prefs)
website = "https://www.forbes.com/advisor/investing/fed-funds-rate-history/"
# path = "C/Users/Admin/Documents/Old D disk/Web scraping/"
driver = webdriver.Chrome(options=chrome_options)
driver.get(website)
driver.maximize_window()

table = driver.find_element(By.XPATH, "//table[@id='footable_336565']")
rows = table.find_elements(By.XPATH, "./tbody/tr")
FOMC_date = []
interest_rate_change = []
federal_funds_rate = []

for row in rows:
    date = row.find_element(By.XPATH, './td[1]').text
    rate_change = row.find_element(By.XPATH, './td[2]').text
    fed_rate = row.find_element(By.XPATH, './td[3]').text

    FOMC_date.append(date)
    interest_rate_change.append(rate_change)
    federal_funds_rate.append(fed_rate)

df = pd.DataFrame({'FOMC_date': FOMC_date, 'interest_rate_change': interest_rate_change, 'federal_funds_rate': federal_funds_rate})
df.to_csv("Fed Rate Hikes 2022-2023 data.csv", index=False, encoding='utf-8-sig')
driver.quit()